Preparation:
- git load ipyleaflet, geopandas, leafmap
- install only once

In [1]:
%%bash

#Matplotlib,Pillow,Pandas,Mamba,Geopandas,Geemap,Leafmap
#sudo pip install protobuf-compiler
apt install -y protobuf-compiler
pip -q install shapely
pip -q install geopy
pip -q install matplotlib
pip -q install pillow
pip -q install xyzservices
pip -q install pandas
pip -q install mamba
pip -q install jupyter_contrib_nbextensions
pip -q install geopandas
pip -q install keplergl
pip -q install python-Levenshtein

#jupyter nbextension install --py --symlink --sys-prefix ipyleaflet
#jupyter nbextension enable --py --sys-prefix ipyleaflet

#jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet

pip -q install ipywidgets
pip -q install geemap xarray_leaflet
pip -q install -U geemap
pip -q install leafmap
pip -q install -U leafmap

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


First excercise: Show me New York with leafmap as a hybrid google map at a zoom of 12 and with a height of 450px and a width of 800px and finally a maximum zoom of 20.

In [2]:
print('Starting first leafmap test...')

import leafmap
#center to New York at 41 degrees north and 74 degrees west
m = leafmap.Map(google_map="HYBRID",center=[40.702557, -74.012318], zoom=12,height="450px", width="800px",max_zoom="20")  #center=[lat, lon]
#show
m

Starting first leafmap test...


Map(center=[40.702557, -74.012318], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Second Excercise: Parse available yellow and green taxi dataset to a variable by streaming and preserve all information from the csv. Use a class, type hints and multithreading. Make sure the data is available and show the min-max-timeframe where your data is available. Prepare adjustment for time ranges that skip already loaded tables. For testing print minimum and maximum available pickup time.

Show available file types for taxis and the minimum and maximum pickup times found:

In [3]:
import read_taxidata

print('Starting first leafmap test...')
td = read_taxidata.TaxiData('/home/benjamin-elias/Proseminar/Jupyterlab/taxi_data/')
here_files = td.get_date_files(2021, 3)
print(here_files)
td.load_add_available(here_files)
print('Minimum time available:')
print(td.get_minimum_available_pickup_time())
print('Maximum time available:')
print(td.get_maximum_available_pickup_time())

Starting first leafmap test...
{'yellow': [datetime.datetime(2021, 3, 1, 0, 0)], 'green': [datetime.datetime(2021, 3, 1, 0, 0)]}
Minimum time available:
2020-01-01 00:00:00
Maximum time available:
2021-07-31 23:59:58


Third Excercise: Parse the taxi zone lookup. Keep it forever.

In [4]:
import read_taxizone

tz = read_taxizone.TaxiZone('/home/benjamin-elias/PycharmProjects/Proseminar Jupyter-Leafmap/taxi_data/taxi+_zone_lookup.csv')
print('Printing zone 29:')
print(tz.get_from_location_id(29))
print('Printing neighbourhood Windsor Terrace:')
print(tz.get_from_neighborhood_name('Windsor Terrace'))

Printing zone 29:
(29, 'Brooklyn', 'Brighton Beach', 'Boro Zone')
Printing neighbourhood Windsor Terrace:
(257, 'Brooklyn', 'Windsor Terrace', 'Boro Zone')


Fourth Excercise: Add the Neighbourhoods layer to the map.

In [6]:
import leafmap
#center to New York at 41 degrees north and 74 degrees west
m = leafmap.Map(google_map="HYBRID",center=[40.702557, -74.012318], zoom=12,height="600px", width="1200px",max_zoom="20")  #center=[lat, lon]
#show
print('Showing current layer names before adding neighbourhoods:')
print(m.get_layer_names())
print('Add geojson layer neighbourhoods:')
m.add_geojson('/home/benjamin-elias/PycharmProjects/Proseminar Jupyter-Leafmap/nyc-neighborhoods.geo.json', layer_name='neighbourhoods', style={}, hover_style={}, style_callback=None, fill_colors=['red'], info_mode='on_hover')
m.layer_opacity('neighbourhoods', 0.3)
print(m.get_layer_names())
m

Showing current layer names before adding neighbourhoods:
['OpenStreetMap', 'Google Satellite']
Add geojson layer neighbourhoods:
['OpenStreetMap', 'Google Satellite', 'neighbourhoods']


Map(center=[40.702557, -74.012318], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Fifth Excercise: Fill all neighbourhoods with random colors and add again.

In [7]:
import leafmap
import random
from typing import List
#center to New York at 41 degrees north and 74 degrees west
m = leafmap.Map(google_map="HYBRID",center=[40.702557, -74.012318], zoom=12,height="600px", width="1200px",max_zoom="20")  #center=[lat, lon]
#show
print('Showing current layer names before adding neighbourhoods:')
print(m.get_layer_names())
print('Random color hex:')
r = lambda: random.randint(0,255)
print('#%02X%02X%02X' % (r(),r(),r()))
print('Random color hex list:')
color_list: List[str] = []
for i in range(0, len(tz.zones)):
    color_list.append('#%02X%02X%02X' % (r(), r(), r()))
print('Add geojson layer neighbourhoods, color is sorted by geojson polygon order:')
m.add_geojson('/home/benjamin-elias/PycharmProjects/Proseminar Jupyter-Leafmap/nyc-neighborhoods.geo.json', layer_name='neighbourhoods', style={}, hover_style={}, style_callback=None, fill_colors=color_list, info_mode='on_hover')
m.layer_opacity('neighbourhoods', 0.3)
print(m.get_layer_names())
m

Showing current layer names before adding neighbourhoods:
['OpenStreetMap', 'Google Satellite']
Random color hex:
#C7B2F5
Random color hex list:
Add geojson layer neighbourhoods, color is sorted by geojson polygon order:
['OpenStreetMap', 'Google Satellite', 'neighbourhoods']


Map(center=[40.702557, -74.012318], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Sixth Excercise: Manually parse the Neighbourhoods layer and calculate the central pinpoint of each, show the pinpoints on the map and oriented on that the neighbourhood name. Make sure you are able to connect the taxi zone information with the geojson for nyc neighborhoods. Maybe correct the taxi zones and use Levenshtein distance to estimate the correct target names first.

First we show how to find the correct taxi zone for each neighborhood. Remind yourself that there are less neighborhoods than taxizones, so that the taxi zones are pretty detailed and cover only one neighborhood. In the end of the first scan not all taxi zones will be used. To have a good filtering system we should scan the geojson with the taxi zones now to make a double join of data.

In [10]:
from read_neighbourhoods import NeighbourhoodTaxiData
nd = NeighbourhoodTaxiData('/home/benjamin-elias/PycharmProjects/Proseminar Jupyter-Leafmap/nyc-neighborhoods.geo.json')

Multiple found: Williamsburg
{(80, 'Brooklyn', 'East Williamsburg', 'Boro Zone'), (256, 'Brooklyn', 'Williamsburg (South Side)', 'Boro Zone'), (255, 'Brooklyn', 'Williamsburg/Williamsburg (North)', 'Boro Zone'), (217, 'Brooklyn', 'South Williamsburg', 'Boro Zone')}
Length analysis showed that this should be correct on the input taxi lookup table:
(255, 'Brooklyn', 'Williamsburg/Williamsburg (North)', 'Boro Zone')

Multiple found: South Williamsburg
{(255, 'Brooklyn', 'Williamsburg/Williamsburg (North)', 'Boro Zone'), (217, 'Brooklyn', 'South Williamsburg', 'Boro Zone')}
Length analysis showed that this should be correct on the input taxi lookup table:
(217, 'Brooklyn', 'South Williamsburg', 'Boro Zone')

Multiple found: East Williamsburg
{(80, 'Brooklyn', 'East Williamsburg', 'Boro Zone'), (255, 'Brooklyn', 'Williamsburg/Williamsburg (North)', 'Boro Zone')}
Length analysis showed that this should be correct on the input taxi lookup table:
(80, 'Brooklyn', 'East Williamsburg', 'Boro Zon

Since the last step we have two lists within NeighborhoodData. Those are the following list of tuples where the index references to another list of polygon lists (that cannot be printed because of size):

In [11]:
for n in nd.neighbourhoodTuples:
    print(n)

(112, 'Brooklyn', 'Greenpoint', 'Boro Zone')
(255, 'Brooklyn', 'Williamsburg/Williamsburg (North)', 'Boro Zone')
(217, 'Brooklyn', 'South Williamsburg', 'Boro Zone')
(80, 'Brooklyn', 'East Williamsburg', 'Boro Zone')
(33, 'Brooklyn', 'Brooklyn Heights', 'Boro Zone')
(65, 'Brooklyn', 'Downtown Brooklyn/MetroTech', 'Boro Zone')
(97, 'Brooklyn', 'Fort Greene', 'Boro Zone')
(49, 'Brooklyn', 'Clinton Hill', 'Boro Zone')
(34, 'Brooklyn', 'Brooklyn Navy Yard', 'Boro Zone')
(17, 'Brooklyn', 'Bedford', 'Boro Zone')
(36, 'Brooklyn', 'Bushwick North (West)', 'Boro Zone')
(37, 'Brooklyn', 'Bushwick South (East)/The Evergreens Cemetery', 'Boro Zone')
(63, 'Brooklyn', 'Cypress Hills', 'Boro Zone')
(77, 'Brooklyn', 'East New York (North)/Pennsylvania Avenue', 'Boro Zone')
(76, 'Brooklyn', 'East New York/New Lots/City Line', 'Boro Zone')
(222, 'Brooklyn', 'Starrett City/Shirley Chisholm State Park', 'Boro Zone')
(40, 'Brooklyn', 'Carroll Gardens', 'Boro Zone')
(181, 'Brooklyn', 'Park Slope', 'Boro Zon

In the last step we calculate the middle pin points of the polygons whom the taxi zone tuples are pointing to right now and add the three lists (taxi zone tuple list, polygon list and centrals list) to a geojson output.

In [ ]:
print('Central point list:')
for c in ntd.central_points():
    print(c)
ntd.to_geojson('/home/benjamin-elias/PycharmProjects/Proseminar Jupyter-Leafmap/taxi_zones.geo.json')

Seventh Excercise: Add a layer to the map that contains the taxi usage over the most recent month parsed above and show that by using lines between the neighbourhood pinpoints. Add an extra variable to save: yellow and green

Eighth Excercise: Add a time widget to the map so you can select date and time. The widget is limited to the minimal and the maximum time found within the folder "taxi_data".

Ninth Excercise: Efficiently parse the time from taxi_data from the datasets available. On change the time frame is checked, when shrinked time sets are popped and removed, when increased, time sets are pushed to front or to the back and drawn to the map. Also upgrade the interconnections between the neighbourhoods within this timeframe.

Tenth Excercise: Add a heatmap containing the colors blue for start destination and red for target destination and draw them as a mixed color into the neighbourhoods into a new layer below the lines layer showing the interconnections. The intensity of the colors should be in comparison to the overall count of starts and arrivals for each neighbourhood.

Eleventh Exercise: Add a time filter widget so we can only see time sets from certain daytimes within the overall time range.

Twelveth Exercise: Add a passenger filter that can limit the passenger count within ranges. Refer minimum and maximum from values found. Repeat that with the taxi color and the vendor ID.

Thirteenth Excercise: Add a filter that can limit the distance moved count within ranges. Refer minimum and maximum from values found.

Fourteenth Excercise: Add a filter that can limit the money spent on the trip within ranges. Refer minimum and maximum from values found.

Fifteenth Excercise: Add a filter that can limit the neighbourhoods either on start (S) or arrival (A) using one string for mode and one string separated with commas to list the neighbourhoods. If anything is empty the filter is off.

Sixteenth Excercise: Add a filter that can select the payment type on the trip. Refer values available.

Seventeenth Excercise: Add tooltips that are highlighted on mouse hover within the borders of each neighbourhood that show the summarized and average data over the csv files always separated on start and arrival point like money spent, payment method, distance traveled and time per distance. The tooltip may contain a barchart showing the average count of starts and arrivals within the chosen daytime.